## Use spectral clustering to find clusters in Political blogs dataset 

###  I aim to analyze the network of political blogs using clustering techniques. The network consists of nodes representing individual blogs, with their connections defined in the file "edges.txt". I plan to conduct clustering experiments with different values of k (2, 5, 10, and 25) to identify distinct groups within the network.

### Once the clusters are formed, I will determine the majority label within each cluster for each value of k. For example, if there are k = 2 clusters with labels {0, 1, 1, 1} and {0, 0, 1}, the majority label for the first cluster would be 1, and for the second cluster, it would be 0.

### I can compare the majority label with the individual labels within each cluster and calculate the mismatch rate for each cluster. For instance, in the aforementioned example, the mismatch rate for the first cluster would be 1/4 (as only the first node differs from the majority), and for the second cluster, it would be 1/3.

### Through this analysis, I aim to gain insights into the clustering structure of political blogs and understand the degree of consensus or divergence within different clusters across varying k values.

In [36]:
import os
import numpy as np
from os.path import abspath, exists
from scipy import sparse
import scipy
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from scipy import stats

In [37]:
def read_political_blogs():
    # read nodes.txt file
    f_path = abspath("data/nodes.txt")
    idx2name = []
    if exists(f_path):
        with open(f_path) as fid:
            for line in fid.readlines():
                name = line.split("\t")
                idx2name.append(name)
    return idx2name
def import_graph():
    # read the graph from 'play_graph.txt' 
    f_path = abspath("data/edges.txt")
    if exists(f_path):
        with open(f_path) as graph_file:
            lines = [line.split() for line in graph_file]
    return np.array(lines).astype(int)

In [38]:
# load the graph
a = import_graph()

# Pre-processing (1. remove nodes with only self edges)
a = a[a[:,0] != a[:,1],]

# Pre-processing (2. remove isolated nodes by reindexing)
unique_nodes = np.unique(a)
index = np.arange(unique_nodes.shape[0]);
index_pairs = {}
for A, B in zip(unique_nodes, index):
    index_pairs[A] = B
new_a = np.copy(a)
for k, v in index_pairs.items(): new_a[a==k] = v
# https://stackoverflow.com/questions/3403973/fast-replacement-of-values-in-a-numpy-array

# Load the political blogs and get the true labels
political_blog_names = read_political_blogs()
true_labels = np.array([line[2] for line in political_blog_names])
true_labels = true_labels.astype('int_')
true_labels_new = true_labels[unique_nodes-1] # exclude isolated nodes

In [39]:
n = max(index) + 1 #1224 nodes
i = new_a[:, 0]
j = new_a[:, 1]
v = np.ones((new_a.shape[0], 1)).flatten()
A = sparse.coo_matrix((v, (i, j)), shape=(n, n))
A = (A + np.transpose(A))
A = sparse.csc_matrix.todense(A) # convert to dense matrix

In [40]:
A.shape

(1224, 1224)

In [41]:
D = np.diag(np.sum(A, axis=1))
L = D - A

In [42]:
# eigendecompoosition
lambd, v = np.linalg.eig(L)
idx = lambd.argsort()
v = v[:, idx]

In [43]:
import warnings
warnings.filterwarnings("ignore")

def get_cluster_labels(v,K):   
    selected_v = v[:, 0:K].real
    kmeans = KMeans(n_clusters=K).fit(selected_v)
    idx = kmeans.labels_
    return idx

In [44]:
k = [2,5,10,25]
for i in k:
    idx = get_cluster_labels(v,i)
    # get the true labels of each cluster
    print("k = %d" % i)
    for j in set(idx): 
        k_j = true_labels_new[idx == j]
        m, count = stats.mode(k_j)
        n = len(k_j)
        mismatch_rate = (k_j != m).sum() / n 
        print("Cluster %d with %d datapoints: mismatch rate %f" % (j, n, mismatch_rate))

k = 2
Cluster 0 with 1116 datapoints: mismatch rate 0.430108
Cluster 1 with 108 datapoints: mismatch rate 0.000000
k = 5
Cluster 0 with 101 datapoints: mismatch rate 0.029703
Cluster 1 with 64 datapoints: mismatch rate 0.000000
Cluster 2 with 836 datapoints: mismatch rate 0.483254
Cluster 3 with 90 datapoints: mismatch rate 0.011111
Cluster 4 with 133 datapoints: mismatch rate 0.000000
k = 10
Cluster 0 with 34 datapoints: mismatch rate 0.000000
Cluster 1 with 810 datapoints: mismatch rate 0.492593
Cluster 2 with 29 datapoints: mismatch rate 0.000000
Cluster 3 with 101 datapoints: mismatch rate 0.009901
Cluster 4 with 42 datapoints: mismatch rate 0.000000
Cluster 5 with 31 datapoints: mismatch rate 0.064516
Cluster 6 with 33 datapoints: mismatch rate 0.000000
Cluster 7 with 20 datapoints: mismatch rate 0.400000
Cluster 8 with 49 datapoints: mismatch rate 0.040816
Cluster 9 with 75 datapoints: mismatch rate 0.040000
k = 25
Cluster 0 with 28 datapoints: mismatch rate 0.000000
Cluster 1 wi

## Tune k and find the number of clusters to achieve a reasonably small mismatch rate. 

By running it with a range of k values from 1 to 40, the mismatch rate is high when k is small (less than 4 or 5). It starts to dropped from that depending on the random seed.

After running it for different random seeds, I find the lowest mismatch rate is 4.8% at k = 7. The mismatch rate is also better and more stable after normalizing the vectors. 

This could imply the political blogs with the same political orientation have denser connections compared to the rest of the network. Hence, the mismatch rate didn't change much after increasing the k-values. 


In [45]:
D = np.diag(1/np.sqrt(np.sum(A, axis=1)).A1)
L = D @ A @ D
L = np.array(L) # covert to array

In [46]:
L.shape

(1224, 1224)

In [47]:
# eigendecompoosition
v, x= np.linalg.eig(L)
idx_sorted = np.argsort(v) # the index of eigenvalue sorted acsending

In [48]:
def get_mode(x):
    values, counts = np.unique(x, return_counts=True)
    m = counts.argmax()
    return values[m], counts[m]

# store the overall mismatch rate for different value of k
overall_dict = {}
# get mismatch rate for k values from 1 to 40
for k in range(1,40,1):
    x_k = x[:, idx_sorted[-k:]] # select the k largest eigenvectors
    x_k = x_k/np.repeat(np.sqrt(np.sum(x_k*x_k, axis=1).reshape(-1, 1)), k, axis=1)
    kmeans = KMeans(n_init = 20, n_clusters=k, random_state = 111).fit(x_k.real)
    c_idx = kmeans.labels_
    # to store the no of mismatched notes/all nodes
    rate = [0,0] 
    for j in set(c_idx): 
        k_j = true_labels_new[c_idx == j]
        m, count = get_mode(k_j) #stats.mode(k_j)
        rate[0] += (k_j != m).sum() # no of mismatch
        rate[1] += len(k_j) # no of nodes in cluster j
#         n = len(k_j)
#         mismatch_rate = (k_j != m).sum() / n 
#         print("Cluster %d %d points: mismatch rate %f" % (j, n, mismatch_rate))

    num= rate[0]
    den= rate[1]
    overall_rate = num/den
    overall_dict[k] = overall_rate
    print("K= %d: mismatch rate %f  (%d,%d) "  % (k, overall_rate, num,den))

print("\n K = %d with lowest mismatch rate %f" % (min(overall_dict, key=overall_dict.get),min(overall_dict.values())))

K= 1: mismatch rate 0.480392  (588,1224) 
K= 2: mismatch rate 0.478758  (586,1224) 
K= 3: mismatch rate 0.478758  (586,1224) 
K= 4: mismatch rate 0.053922  (66,1224) 
K= 5: mismatch rate 0.050654  (62,1224) 
K= 6: mismatch rate 0.051471  (63,1224) 
K= 7: mismatch rate 0.048203  (59,1224) 
K= 8: mismatch rate 0.052288  (64,1224) 
K= 9: mismatch rate 0.054739  (67,1224) 
K= 10: mismatch rate 0.061275  (75,1224) 
K= 11: mismatch rate 0.059641  (73,1224) 
K= 12: mismatch rate 0.053922  (66,1224) 
K= 13: mismatch rate 0.061275  (75,1224) 
K= 14: mismatch rate 0.058824  (72,1224) 
K= 15: mismatch rate 0.059641  (73,1224) 
K= 16: mismatch rate 0.058007  (71,1224) 
K= 17: mismatch rate 0.063725  (78,1224) 
K= 18: mismatch rate 0.059641  (73,1224) 
K= 19: mismatch rate 0.063725  (78,1224) 
K= 20: mismatch rate 0.065359  (80,1224) 
K= 21: mismatch rate 0.061275  (75,1224) 
K= 22: mismatch rate 0.062908  (77,1224) 
K= 23: mismatch rate 0.064542  (79,1224) 
K= 24: mismatch rate 0.062092  (76,1224)